In [1]:
from pathlib import Path

import numpy as np
import scipy
from keras import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import Precision, Recall
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Load data

In [2]:
paths_list = Path("./102flowers/resized_512").glob('**/*.jpg')
X = np.array([imread(str(path)) for path in paths_list])

In [3]:
mat = scipy.io.loadmat("./102flowers/imagelabels.mat")
y = mat['labels'][0, :]
num_classes = len(np.unique(y))  # from 1 to 102

In [4]:
X, y = shuffle(X, y, random_state=33)
y_onehot = np.zeros((y.shape[0], num_classes))
for i, val in enumerate(y):
    y_onehot[i, val - 1] = 1

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.5, random_state=33)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=33)

In [6]:
del X
del y
del y_onehot

# VGG19

In [7]:
keras.applications.vgg19.VGG19().summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [23]:
vgg19 = keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(512, 512, 3),
    classes=num_classes,
    classifier_activation='softmax')

In [24]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 512, 512, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 512, 512, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 256, 256, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 256, 256, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 128, 128, 128)     0     

In [25]:
for layer in vgg19.layers:
    layer.trainable = False

In [26]:
out = Flatten()(vgg19.output)
out = Dense(4096, activation='relu', name="fc1")(out)
out = Dense(4096, activation='relu', name="fc2")(out)
predictions = Dense(num_classes, activation='softmax', name="predictions")(out)
model = Model(inputs=vgg19.input, outputs=predictions)

In [27]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 512, 512, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 512, 512, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 256, 256, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 256, 256, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 128, 128, 128)     0   

In [29]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', Precision(), Recall(),
                       'categorical_accuracy', 'categorical_crossentropy'])

In [ ]:
model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val),
          batch_size=128, shuffle=True, epochs=50)

Epoch 1/50


# YOLOv5